In [39]:
import requests
from bs4 import BeautifulSoup
import re
import sys
import argparse
from selenium import webdriver
from io import BytesIO

try:
    import pytesseract
except:
    !pip install pytesseract
    import pytesseract

try:
    from PIL import Image
except:
    !pip install PIL
    from PIL import Image

try:
    from selenium.webdriver import Chrome
except:
    !pip install selenium
    from selenium.webdriver import Chrome

pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files\Tesseract-OCR\\tesseract.exe'

In [73]:
driver = Chrome('chromedriver_win32/chromedriver.exe')
link = 'http://www.sunat.gob.pe/cl-ti-itmrconsruc/FrameCriterioBusquedaMovil.jsp'
driver.get(link)

In [74]:
driver.implicitly_wait(10)
element = driver.find_element_by_name('imagen')

pos = element.location
size = element.size
png = driver.get_screenshot_as_png()

img = Image.open(BytesIO(png))
left = pos['x']
top = pos['y']
right = pos['x'] + size['width']
bottom = pos['y'] + size['height']
img = img.crop((left, top, right, bottom))

captcha = pytesseract.image_to_string(img)

In [75]:
ruc = '10749304389'

driver.find_element_by_id('txtRuc').send_keys(ruc)
driver.find_element_by_id('txtCodigo').send_keys(captcha)
driver.find_element_by_id('btnAceptar').click()

driver.implicitly_wait(10)
html = driver.find_element_by_class_name('list-group').get_attribute('innerHTML')

In [76]:
content = BeautifulSoup(html, 'html.parser')

In [77]:
inforuc = {}

info = content.select('.list-group-item')

for elem in info:
    key = elem.find(class_='col-sm-5').find('h4').text[:-1].strip() # [:-1] para eliminar los dos puntos
    data = elem.find(class_='col-sm-7').text.strip()
    inforuc[key] = data

In [78]:
inforuc

{'RUC': '10749304389 - TUYA RODRIGUEZ PAULO CESAR',
 'Tipo Contribuyente': 'PERSONA NATURAL SIN NEGOCIO',
 'Tipo de Documento': 'DNI  74930438 \n\t\t\t\t\t\t            \n\t\t\t\t\t\t            \t - TUYA RODRIGUEZ, PAULO CESAR',
 'Nombre Comercial': '-',
 'Fecha de Inscripción': '17/05/2018',
 'Estado': 'ACTIVO',
 'Condición': 'HABIDO',
 'Domicilio Fiscal': '-',
 'Actividad(es) Económica(s)': 'Principal        - CIIU 93098 - OTRAS ACTIVID.DE TIPO SERVICIO NCP',
 'Comprobantes de Pago c/aut. de impresión (F. 806 u 816)': 'NINGUNO',
 'Sistema de Emisión Electrónica': 'RECIBOS POR HONORARIOS     AFILIADO DESDE 21/05/2018',
 'Afiliado al PLE desde': '-',
 'Padrones': 'NINGUNO'}

In [79]:
driver.find_element_by_class_name('btnNuevaConsulta').click()

In [80]:
driver.quit()